# Coinbase Pro Portfolio Optimization

Adapted from original publication by [Brick Pop](https://medium.com/@brickpop):

[Finding top crypto portfolios with Tensorflow and Matrix calculus](https://medium.com/stack-me-up/crypto-portfolio-optimization-with-python-and-tensorflow-an-approach-aa504578c799)

In [1]:
import json
import requests
import pandas as pd
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# GLOBAL
# Coins currently in portfolio
#coins = ["BCH", "BTC", "DOGE", "ETH", "LTC", "USDT"]
coins = ["ETC", "ETH", "BTC", "MKR", "LTC"]
# Adjust these two variables based on your currency and
# time frame you are looking to trade in.
primary_quote = 'BTC'
quote_currecy = 'USD'
granularity = 86400 # {60, 300, 900, 3600, 21600, 86400}

# Not sure a start date is needed?
start_date = '2020-08-17' # 2021-02-01

days_ago_to_fetch = 400  # see also filter_history_by_date()
coin_history = {}
hist_length = 0
average_returns = {}
cumulative_returns = {}

def fetch_all():
  for coin in coins:
    coin_history[coin] = fetch_history(coin)

def fetch_history(coin):
  print('Fetching: ' + coin)
  #endpoint_url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym=USD&limit={:d}".format(coin, days_ago_to_fetch)
  endpoint_url = f'https://api.pro.coinbase.com/products/{coin}-{quote_currecy}/candles?granularity={granularity}'
  response = requests.get(endpoint_url)
  hist = pd.DataFrame(json.loads(response.text), columns=['time', 'low', 'high', 'open', 'close', 'volume'])
  hist = index_history(hist)
  hist = filter_history_by_date(hist)
  return hist

def index_history(hist):
  # index by date so we can easily filter by a given timeframe
  hist = hist.set_index(['time'])
  hist.index = pd.to_datetime(hist.index, unit='s')
  return hist

def filter_history_by_date(hist):
  result = hist # customize here
  # result = hist[hist.index.year >= 2017]
  # result = result[result.index.day == 1] # every first of month, etc.
  return result

fetch_all()
hist_length = len(coin_history[coins[0]])

coin_history[primary_quote]

Fetching: ETC
Fetching: ETH
Fetching: BTC
Fetching: MKR
Fetching: LTC


,low,high,open,close,volume
time,,,,,
2021-06-14,38808.86,39249.00,39015.24,38871.34,288.033274
2021-06-13,34780.57,39396.00,35557.32,39015.24,15748.962040
2021-06-12,34635.47,37448.00,37340.08,35557.33,14516.545563
2021-06-11,35944.00,37695.00,36694.91,37338.44,14142.193500
2021-06-10,35800.00,38425.67,37404.75,36694.05,19853.814189
...,...,...,...,...,...
2020-08-23,11525.62,11715.00,11672.92,11650.01,5516.170482
2020-08-22,11370.00,11693.16,11529.99,11672.93,7946.024653
2020-08-21,11480.00,11884.99,11864.26,11529.22,14865.378759


In [3]:
# Calculate returns and excess returns

def add_all_returns():
  for coin in coins:
    hist = coin_history[coin]
    hist['return'] = (hist['close'] - hist['open']) / hist['open']
    average = hist["return"].mean()
    average_returns[coin] = average
    cumulative_returns[coin] = (hist["return"] + 1).prod() - 1
    hist['excess_return'] = hist['return'] - average
    coin_history[coin] = hist

add_all_returns()

# display data
print(cumulative_returns)
coin_history[primary_quote]

{'ETC': 6.818914446640908, 'ETH': 4.9351985453016365, 'BTC': 2.2453979343838335, 'MKR': 3.514610007571341, 'LTC': 1.6133233728423448}


,low,high,open,close,volume,return,excess_return
time,,,,,,,
2021-06-14,38808.86,39249.00,39015.24,38871.34,288.033274,-0.003688,-0.008491
2021-06-13,34780.57,39396.00,35557.32,39015.24,15748.962040,0.097249,0.092447
2021-06-12,34635.47,37448.00,37340.08,35557.33,14516.545563,-0.047744,-0.052546
2021-06-11,35944.00,37695.00,36694.91,37338.44,14142.193500,0.017537,0.012735
2021-06-10,35800.00,38425.67,37404.75,36694.05,19853.814189,-0.019000,-0.023802
...,...,...,...,...,...,...,...
2020-08-23,11525.62,11715.00,11672.92,11650.01,5516.170482,-0.001963,-0.006765
2020-08-22,11370.00,11693.16,11529.99,11672.93,7946.024653,0.012397,0.007595
2020-08-21,11480.00,11884.99,11864.26,11529.22,14865.378759,-0.028239,-0.033042


In [4]:
# Drop coins that don't have enough data
new_coins = []
new_coin_hist = {}

for coin in coins:
    #print(f'coin: {coin}\tlen: {len(coin_history[coin])}')
    if (len(coin_history[coin]) == hist_length):
        new_coins.append(coin)
        new_coin_hist[coin] = coin_history[coin]

coins = new_coins
coin_history = new_coin_hist

#for coin in coins:
    #print(f'{coin}\t{len(coin_history[coin])}')

In [5]:
# Excess matrix

excess_matrix = np.zeros((hist_length, len(coins)))

for i in range(0, hist_length):
  for idx, coin in enumerate(coins):
    excess_matrix[i][idx] = coin_history[coin].iloc[i]['excess_return']

# Display

pretty_matrix = pd.DataFrame(excess_matrix, columns = coins)
pretty_matrix.index = coin_history[coins[0]].index

pretty_matrix

,ETC,ETH,BTC,MKR,LTC
time,,,,,
2021-06-14,-0.020285,-0.012260,-0.008491,-0.016597,-0.011568
2021-06-13,0.058403,0.050672,0.092447,0.062227,0.051226
2021-06-12,-0.037683,-0.000716,-0.052546,-0.004797,-0.011690
2021-06-11,-0.028975,-0.054734,0.012735,-0.067737,-0.038371
2021-06-10,-0.083106,-0.061209,-0.023802,-0.075019,-0.030324
...,...,...,...,...,...
2020-08-23,-0.006488,-0.019352,-0.006765,-0.012528,0.000274
2020-08-22,0.013670,0.011968,0.007595,0.054828,0.014773
2020-08-21,-0.058932,-0.074679,-0.033042,-0.095467,-0.064742


In [6]:
# Variance co-variance matrix

product_matrix = np.matmul(excess_matrix.transpose(), excess_matrix)
var_covar_matrix = product_matrix / hist_length

# Display

pretty_matrix = pd.DataFrame(var_covar_matrix, columns=coins, index=coins)

pretty_matrix

,ETC,ETH,BTC,MKR,LTC
ETC,0.007868,0.002789,0.001618,0.002909,0.003631
ETH,0.002789,0.003423,0.001769,0.003242,0.002976
BTC,0.001618,0.001769,0.001754,0.001627,0.002058
MKR,0.002909,0.003242,0.001627,0.006739,0.002781
LTC,0.003631,0.002976,0.002058,0.002781,0.004223


In [7]:
# Standard Deviation

std_deviations = np.zeros((len(coins), 1))

for idx, coin in enumerate(coins):
  std_deviations[idx][0] = np.std(coin_history[coin]['return'])
  
# Display

pretty_matrix = pd.DataFrame(std_deviations, columns=['Std Dev'], index=coins)

pretty_matrix

,Std Dev
ETC,0.088701
ETH,0.058509
BTC,0.041877
MKR,0.082091
LTC,0.064985


In [8]:
# Std Deviation products matrix

sdev_product_matrix = np.matmul(std_deviations, std_deviations.transpose())


# Display

pretty_matrix = pd.DataFrame(sdev_product_matrix, columns=coins, index=coins)

pretty_matrix

,ETC,ETH,BTC,MKR,LTC
ETC,0.007868,0.005190,0.003715,0.007282,0.005764
ETH,0.005190,0.003423,0.002450,0.004803,0.003802
BTC,0.003715,0.002450,0.001754,0.003438,0.002721
MKR,0.007282,0.004803,0.003438,0.006739,0.005335
LTC,0.005764,0.003802,0.002721,0.005335,0.004223


In [9]:
# Correlation matrix

correlation_matrix = var_covar_matrix / sdev_product_matrix

# Display

pretty_matrix = pd.DataFrame(correlation_matrix, columns=coins, index=coins)

pretty_matrix

,ETC,ETH,BTC,MKR,LTC
ETC,1.000000,0.537469,0.435516,0.399439,0.629974
ETH,0.537469,1.000000,0.721801,0.674922,0.782646
BTC,0.435516,0.721801,1.000000,0.473387,0.756068
MKR,0.399439,0.674922,0.473387,1.000000,0.521214
LTC,0.629974,0.782646,0.756068,0.521214,1.000000


In [10]:
# Optimize weights to minimize variance

def minimize_volatility():

  # Define the model
  # Portfolio Volatility = Sqrt (Transpose (Wt.SD) * Correlation Matrix * Wt. SD)

  coin_weights = tf.Variable(np.full((len(coins), 1), 1.0 / len(coins))) # our variables
  weighted_std_devs = tf.multiply(coin_weights, std_deviations)

  product_1 = tf.transpose(weighted_std_devs)
  product_2 = tf.matmul(product_1, correlation_matrix)
  
  portfolio_variance = tf.matmul(product_2, weighted_std_devs)
  portfolio_volatility = tf.sqrt(tf.reduce_sum(portfolio_variance))


  # Run
  learn_rate = 0.01
  steps = 5000
  
  init = tf.global_variables_initializer()

  # Training using Gradient Descent to minimize variance
  train_step = tf.train.GradientDescentOptimizer(learn_rate).minimize(portfolio_volatility)

  with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
      sess.run(train_step)
      if i % 1000 == 0 :
        print("[round {:d}]".format(i))
        print("Weights", coin_weights.eval())
        print("Volatility: {:.2f}%".format(portfolio_volatility.eval() * 100))
        print("")
        
    return coin_weights.eval()

weights = minimize_volatility()

pretty_weights = pd.DataFrame(weights * 100, index = coins, columns = ["Weight %"])
pretty_weights

[round 0]
Weights [[0.19931208]
 [0.19948086]
 [0.19967734]
 [0.19936757]
 [0.19942713]]
Volatility: 5.45%

[round 1000]
Weights [[-0.00059522]
 [-0.00041544]
 [-0.00024796]
 [-0.00052255]
 [-0.00046091]]
Volatility: 0.01%

[round 2000]
Weights [[-0.00059522]
 [-0.00041544]
 [-0.00024796]
 [-0.00052255]
 [-0.00046091]]
Volatility: 0.01%

[round 3000]
Weights [[-0.00059522]
 [-0.00041544]
 [-0.00024796]
 [-0.00052255]
 [-0.00046091]]
Volatility: 0.01%

[round 4000]
Weights [[-0.00059522]
 [-0.00041544]
 [-0.00024796]
 [-0.00052255]
 [-0.00046091]]
Volatility: 0.01%



,Weight %
ETC,0.012801
ETH,0.008935
BTC,0.005333
MKR,0.011238
LTC,0.009912


In [11]:
# Optimize weights to minimize variance

def minimize_volatility():

  # Define the model
  # Portfolio Volatility = Sqrt (Transpose (Wt.SD) * Correlation Matrix * Wt. SD)

  coin_weights = tf.Variable(np.full((len(coins), 1), 1.0 / len(coins))) # our variables
  weighted_std_devs = tf.multiply(coin_weights, std_deviations)

  product_1 = tf.transpose(weighted_std_devs)
  product_2 = tf.matmul(product_1, correlation_matrix)
  
  portfolio_variance = tf.matmul(product_2, weighted_std_devs)
  portfolio_volatility = tf.sqrt(tf.reduce_sum(portfolio_variance))

  # Constraints: sum([0..1, 0..1, ...]) = 1

  lower_than_zero = tf.greater( np.float64(0), coin_weights )
  zero_minimum_op = coin_weights.assign( tf.where (lower_than_zero, tf.zeros_like(coin_weights), coin_weights) )

  greater_than_one = tf.greater( coin_weights, np.float64(1) )
  unity_max_op = coin_weights.assign( tf.where (greater_than_one, tf.ones_like(coin_weights), coin_weights) )

  result_sum = tf.reduce_sum(coin_weights)
  unity_sum_op = coin_weights.assign(tf.divide(coin_weights, result_sum))
  
  constraints_op = tf.group(zero_minimum_op, unity_max_op, unity_sum_op)

  # Run
  learning_rate = 0.01
  steps = 5000
  
  init = tf.global_variables_initializer()

  # Training using Gradient Descent to minimize variance
  optimize_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(portfolio_volatility)

  with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
      sess.run(optimize_op)
      sess.run(constraints_op)
      if i % 2500 == 0 :
        print("[round {:d}]".format(i))
        print("Weights", coin_weights.eval())
        print("Volatility: {:.2f}%".format(portfolio_volatility.eval() * 100))
        print("")
        
    sess.run(constraints_op)
    return coin_weights.eval()

weights = minimize_volatility()

pretty_weights = pd.DataFrame(weights * 100, index = coins, columns = ["Weight %"])
pretty_weights

[round 0]
Weights [[0.1998587 ]
 [0.20002794]
 [0.20022496]
 [0.19991434]
 [0.19997406]]
Volatility: 5.47%

[round 2500]
Weights [[0.        ]
 [0.        ]
 [1.00169213]
 [0.        ]
 [0.        ]]
Volatility: 4.19%



,Weight %
ETC,0.0
ETH,0.0
BTC,100.0
MKR,0.0
LTC,0.0


In [12]:
# Optimize weights to maximize return/risk

import time
start = time.time()

def maximize_sharpe_ratio():
  
  # Define the model
  
  # 1) Variance
  
  coin_weights = tf.Variable(tf.random_uniform((len(coins), 1), dtype=tf.float64)) # our variables
  weighted_std_devs = tf.multiply(coin_weights, std_deviations)
  
  product_1 = tf.transpose(weighted_std_devs)
  product_2 = tf.matmul(product_1, correlation_matrix)
  
  portfolio_variance = tf.matmul(product_2, weighted_std_devs)
  portfolio_volatility = tf.sqrt(tf.reduce_sum(portfolio_variance))

  
  # 2) Return
  
  returns = np.full((len(coins), 1), 0.0) # same as coin_weights
  for coin_idx in range(0, len(coins)):
    returns[coin_idx] = cumulative_returns[coins[coin_idx]]
  
  portfolio_return = tf.reduce_sum(tf.multiply(coin_weights, returns))
  
  # 3) Return / Risk
  
  sharpe_ratio = tf.divide(portfolio_return, portfolio_volatility)
  
  # Constraints
  
  # all values positive, with unity sum
  weights_sum = tf.reduce_sum(coin_weights)
  constraints_op = coin_weights.assign(tf.divide(tf.abs(coin_weights), tf.abs(weights_sum) ))
  
  # Run
  learning_rate = 0.0001
  learning_rate = 0.0015
  steps = 10000
  
  # Training using Gradient Descent to minimize cost
  
  optimize_op = tf.train.GradientDescentOptimizer(learning_rate, use_locking=True).minimize(tf.negative(sharpe_ratio))
  #2# optimize_op = tf.train.AdamOptimizer(learning_rate, use_locking=True).minimize(tf.negative(sharpe_ratio))
  #3# optimize_op = tf.train.AdamOptimizer(learning_rate=0.00005, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False).minimize(tf.negative(sharpe_ratio))
  #4# optimize_op = tf.train.AdagradOptimizer(learning_rate=0.01, initial_accumulator_value=0.1, use_locking=False).minimize(tf.negative(sharpe_ratio))
  
  
  init = tf.global_variables_initializer()
  
  with tf.Session() as sess:
    ratios = np.zeros(steps)
    returns = np.zeros(steps)
    sess.run(init)
    for i in range(steps):
      sess.run(optimize_op)
      sess.run(constraints_op)
      ratios[i] = sess.run(sharpe_ratio)
      returns[i] = sess.run(portfolio_return) * 100
      if i % 2000 == 0 : 
        sess.run(constraints_op)
        print("[round {:d}]".format(i))
        #print("Coin weights", sess.run(coin_weights))
        print("Volatility {:.2f} %".format(sess.run(portfolio_volatility)))
        print("Return {:.2f} %".format(sess.run(portfolio_return)*100))
        print("Sharpe ratio", sess.run(sharpe_ratio))
        print("")
    
    sess.run(constraints_op)
    # print("Coin weights", sess.run(coin_weights))
    print("Volatility {:.2f} %".format(sess.run(portfolio_volatility)))
    print("Return {:.2f} %".format(sess.run(portfolio_return)*100))
    print("Sharpe ratio", sess.run(sharpe_ratio))
    return sess.run(coin_weights)

weights = maximize_sharpe_ratio()

print("Took {:f}s to complete".format(time.time() - start))
pretty_weights = pd.DataFrame(weights * 100, index = coins, columns = ["Weight %"])

pretty_weights

[round 0]
Volatility 0.06 %
Return 325.58 %
Sharpe ratio 53.89852036378944

[round 2000]
Volatility 0.06 %
Return 549.48 %
Sharpe ratio 91.04997132599031

[round 4000]
Volatility 0.06 %
Return 546.77 %
Sharpe ratio 90.72360076845548

[round 6000]
Volatility 0.06 %
Return 546.77 %
Sharpe ratio 90.72591726639926

[round 8000]
Volatility 0.06 %
Return 548.14 %
Sharpe ratio 90.87728075644068

Volatility 0.06 %
Return 523.72 %
Sharpe ratio 87.76484015771433
Took 22.816048s to complete


,Weight %
ETC,31.173236
ETH,58.617893
BTC,0.756887
MKR,2.583342
LTC,6.868642
